In [1]:
import mlflow

mlflow.__version__

'2.21.3'

In [2]:
from src.utils.folder_operations import get_project_root

# set mlflow tracking uri
mlflow.set_tracking_uri(
    (get_project_root() / 'mlflow_new/mlruns').as_uri()
)

# Python Model Context

In [3]:
import pandas as pd

In [4]:
class CustomModel(mlflow.pyfunc.PythonModel):

    def predict(self, context, model_input, params=None):
        print("Context Artifacts: ", context.artifacts)
        print("Context Model Configurations: ", context.model_config)
        return model_input.apply(lambda x: x * 2)
    
    def additional_method(self):
        print("Running an additional method")

/Users/caiosainvallio/projects/mlflow_new/.venv/lib/python3.13/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [5]:
with mlflow.start_run(run_name="class_model_context") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model("model", python_model=CustomModel())

52537b7ed7834491bb80bd83a59f5fac


2025/04/21 21:43:53 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/04/21 21:43:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [6]:
model_uri = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [7]:
x_new = pd.DataFrame([1, 2, 3])
prediction = loaded_model.predict(x_new)

Context Artifacts:  {}
Context Model Configurations:  None


# Model Artifacts

In [12]:
with mlflow.start_run(run_name="class_model_context") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model("model",
                            python_model=CustomModel(),
                            artifacts={"my_file": "src/utils/run_batch.py"})

c49f34970ca740d0ab4f7352b3cb41ff


2025/04/21 21:49:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/04/21 21:49:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [13]:
# load the model
model_uri = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [14]:
loaded_model.predict(x_new)

Context Artifacts:  {'my_file': '/Users/caiosainvallio/projects/mlflow_new/mlruns/0/c49f34970ca740d0ab4f7352b3cb41ff/artifacts/model/artifacts/run_batch.py'}
Context Model Configurations:  None


,0
0,2
1,4
2,6


# Model Configurations

In [15]:
with mlflow.start_run(run_name="class_model_context") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model(
        "model",
        python_model=CustomModel(),
        model_config={"model_name": "My Custom Model"}
    )

7dec9a9969b3473da878f0bd1874d464


2025/04/21 21:51:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/04/21 21:51:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
# load the model
model_uri = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [17]:
loaded_model.predict(x_new)

Context Artifacts:  {}
Context Model Configurations:  {'model_name': 'My Custom Model'}


,0
0,2
1,4
2,6


# Load context

In [18]:
class CustomModel(mlflow.pyfunc.PythonModel):

    def predict(self, context, model_input, params=None):
        model_name = context.model_config.get("model_name", "default")
        print("Getting predictions from the model {}".format(model_name))
        print("additional inputs: ")
        print(self.df.head())
        return model_input.apply(lambda x: x * 2)
    
    def additional_method(self):
        print("Running an additional method")

    def load_context(self, context):
        print("loading context....")
        import time 
        time.sleep(5)
        artifact = context.artifacts.get("predictions")
        self.df = pd.read_csv(artifact)

/Users/caiosainvallio/projects/mlflow_new/.venv/lib/python3.13/site-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [26]:
with mlflow.start_run(run_name="class_model_context") as run:
    print(run.info.run_id)
    mlflow.pyfunc.log_model(
        "model",
        python_model=CustomModel(),
        artifacts={
            "predictions":(get_project_root() / 'mlflow_new/artifacts_example/predictions.csv').as_uri()
            },
        model_config={"model_name": "Classifier"}
    )

e8f4d69f995a4e27b450dd4ef56977cf


2025/04/21 21:57:21 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/3m/pn_mgq3x5b315qvdmh8qr6jw0000gn/T/tmpepv_viz1/model, flavor: python_function). Fall back to return ['cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/04/21 21:57:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/04/21 21:57:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [27]:
# load the model
model_uri = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(model_uri)

loading context....


EmptyDataError: No columns to parse from file

In [28]:
predictions = loaded_model.predict(x_new)

Context Artifacts:  {}
Context Model Configurations:  {'model_name': 'My Custom Model'}
